### Declare Global Variable

In [6]:
RAW_DATA = "./raw-data"
PROCESSED_DATA = "./processed-data"
STATION = "./station"

batch_size = 100
num_epochs = 30
learning_rate = 0.00003

### Load Data

In [7]:
# Load Data
import pandas as pd
import numpy as np
def loadData(filename = "modelSequence.csv"):


    dataset = pd.read_csv('{}/modelSequence.csv'.format(PROCESSED_DATA), header = 0)
    dataset.set_index(dataset.columns[0], inplace=True)
    # print(dataset[:5])

    # Convert to numpy array
    np_dataset = np.array(dataset)
    # print(np_dataset.shape) # 40320, 6
    # print(np_dataset.shape[0]) # 40320
    # print(np_dataset.shape[1]) # 6
    # print(np_dataset[:5])


    # divide the dataset into three categories: train, validation, test
    first = int(np_dataset.shape[0] * 0.7)
    second = first + int(np_dataset.shape[0] * 0.2)

    train_data = np_dataset[:first]
    # print(train_data.shape)
    validation_data = np_dataset[first:second]
    # print(validation_data.shape)
    test_data = np_dataset[second:]
    # print(test_data.shape)
    return train_data, validation_data, test_data

train_data, validation_data, test_data = loadData()

### Declare Dataset Class

In [8]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
%matplotlib inline

In [9]:
class Dataset(Dataset):
    def __init__(self, data, mode, mean=None, std=None, seq_len=480, target_delay=24, stride=5, normalize=True):
        self.mode = mode
        self.seq_len = seq_len
        self.target_delay = target_delay
        self.stride = stride
        self.data = data
        if mode == 'train':
            assert (mean is None) and (std is None), \
                "Argument should be None on train mode"
            self.mean = np.mean(data, axis=0)
            self.std = np.std(data, axis=0) 
        else:
            assert (mean is not None) and (std is not None), \
                "You can use mean/std from train_data"
            self.mean = mean
            self.std = std
        
        if normalize:
            self.data = (self.data - self.mean) / self.std
    
    def __getitem__(self, index):
        index = index * self.stride
        sequence = self.data[index:index + self.seq_len, :]
        # print(sequence.shape) (480,9)
        target = self.data[index+self.seq_len+self.target_delay-1, 0]
        target = np.expand_dims(target, axis=0)
        # print(target.shape) (1,)
        return sequence, target
    
    def __len__(self):
        max_idx = len(self.data) - self.seq_len - self.target_delay
        num_of_idx = max_idx // self.stride
        return num_of_idx

### Mount the data into Loaders

In [10]:
train_data = Dataset(train_data, 'train', mean=None, std=None)
val_data = Dataset(validation_data, 'val', mean=train_data.mean, std=train_data.std)
test_data = Dataset(test_data, 'test', mean=train_data.mean, std=train_data.std)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, drop_last=True)

### Define baseline evaluation function

- Sequence is 480 hours data
- Prediction is the temperature after 24 hours later
- Baseline performance means here is the temperature at the last point of the Sequence

In [11]:
def eval_baseline(data_loader, criterion):
    total_loss = 0
    cnt = 0
    for step, (sequence, target) in enumerate(data_loader):
        
        # print("seq", sequence.shape) torch.Size([100, 480, 9]) # batch_size = 100
        pred = sequence[:, -1:, :1]
        # print("pred", pred.shape) # torch.Size([100, 1, 1])
        pred = torch.squeeze(pred, 1)
        # print("pred", pred.shape) # torch.Size([100, 1])
        # print("tar", target.shape) torch.Size([100, 1])
        loss = criterion(pred, target)
            
        total_loss += loss
        cnt += 1
    avrg_loss = total_loss / cnt
    print('Baseline Average Loss: {:.4f}'.format(avrg_loss))
    return avrg_loss.item()

In [12]:
baseline_loss = eval_baseline(test_loader, nn.MSELoss())

Baseline Average Loss: 0.0852


In [13]:
for i in range(15):
    data_idx = np.random.randint(len(test_data))
    pred = test_data[data_idx][0][-1, 0]
    pred = pred * test_data.std[0] + test_data.mean[0]  # un-normalization
    target = test_data[data_idx][1][0] * test_data.std[0] + test_data.mean[0] # un-normalization
    print('Predicted Temp: {:.1f} / Actual Temp: {:.1f}'.format(pred, target))

Predicted Temp: 4.8 / Actual Temp: 8.8
Predicted Temp: 25.9 / Actual Temp: 25.2
Predicted Temp: 21.4 / Actual Temp: 21.1
Predicted Temp: 28.3 / Actual Temp: 20.1
Predicted Temp: 9.8 / Actual Temp: 11.1
Predicted Temp: -4.1 / Actual Temp: 0.3
Predicted Temp: 9.7 / Actual Temp: 11.3
Predicted Temp: 6.0 / Actual Temp: 6.5
Predicted Temp: 20.2 / Actual Temp: 21.5
Predicted Temp: 5.2 / Actual Temp: 10.0
Predicted Temp: 27.8 / Actual Temp: 25.8
Predicted Temp: 7.7 / Actual Temp: 9.1
Predicted Temp: 18.1 / Actual Temp: 15.6
Predicted Temp: 10.9 / Actual Temp: 16.2
Predicted Temp: 21.5 / Actual Temp: 22.3


### Define LSTM

In [14]:
class SimpleLSTM(nn.Module):
    def __init__(self, input_size = 7, output_size = 1, hidden_size=100, num_layers=1):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True) 
        self.fc = nn.Linear(self.hidden_size, self.output_size) 

    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden, cell
    
    def forward(self, x):
        # hidden, cell state init
        h, c = self.init_hidden(x.size(0))
        h, c = h.to(x.device), c.to(x.device)
        out, (h, c) = self.lstm(x, (h, c))     
        final_output = self.fc(out[:, -1:, :])     
        final_output = torch.squeeze(final_output, dim = 1) # shape (100,1)

        return final_output

### Define Train/Validation/Test Functions

In [15]:
def train(num_epochs, model, data_loader, criterion, optimizer, saved_dir, val_every, device):
    print('Start training..')
    best_loss = 9999999
    for epoch in range(num_epochs):
        for step, (sequence, target) in enumerate(data_loader):
            sequence = sequence.type(torch.float32)
            target = target.type(torch.float32)
            sequence, target = sequence.to(device), target.to(device)

            outputs = model(sequence)  
            loss = criterion(outputs, target)     
            optimizer.zero_grad() 
            loss.backward()       
            optimizer.step()       
            
            if (step + 1) % 25 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                    epoch+1, num_epochs, step+1, len(train_loader), loss.item()))
                
        if (epoch + 1) % val_every == 0:
            avrg_loss = validation(epoch + 1, model, val_loader, criterion, device)
            if avrg_loss < best_loss:
                print('Best performance at epoch: {}'.format(epoch + 1))
                print('Save model in', saved_dir)
                best_loss = avrg_loss
                save_model(model, saved_dir)

In [16]:
def validation(epoch, model, data_loader, criterion, device):
    print('Start validation #{}'.format(epoch))
    model.eval()
    with torch.no_grad():
        total_loss = 0
        cnt = 0
        for step, (sequence, target) in enumerate(data_loader):
            sequence = sequence.type(torch.float32)
            target = target.type(torch.float32)
            sequence, target = sequence.to(device), target.to(device)

            outputs = model(sequence)  
            loss = criterion(outputs, target)

            total_loss += loss
            cnt += 1
        avrg_loss = total_loss / cnt
        print('Validation #{}  Average Loss: {:.4f}'.format(epoch, avrg_loss))
    model.train()
    return avrg_loss

In [17]:
def test(model, data_loader, criterion, baseline_loss, device):
    print('Start test..')
    model.eval()
    with torch.no_grad():
        total_loss = 0
        cnt = 0
        for step, (sequence, target) in enumerate(data_loader):
            sequence = sequence.type(torch.float32)
            target = target.type(torch.float32)
            sequence, target = sequence.to(device), target.to(device)
            # print(sequence.shape) 100,480,9
            outputs = model(sequence) 
            loss = criterion(outputs, target)   
            total_loss += loss
            cnt += 1
        avrg_loss = total_loss / cnt
        print('Test  Average Loss: {:.4f}  Baseline Loss: {:.4f}'.format(avrg_loss, baseline_loss))
        
    if avrg_loss < baseline_loss:
        print('Above baseline performance!')
    else:
        print('Below baseline performance')

### Train and Save Model

In [18]:
def save_model(model, saved_dir, file_name='best_model.pt'):
    import os
    os.makedirs(saved_dir, exist_ok=True)
    check_point = {
        'net': model.state_dict()
    }
    output_path = os.path.join(saved_dir, file_name)
    torch.save(check_point, output_path)

In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu" 

In [20]:
torch.manual_seed(7777) 
model = SimpleLSTM()          
model = model.to(device)
criterion = nn.MSELoss()    
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)     
val_every = 1
saved_dir = './saved/LSTM'

In [21]:
train(num_epochs, model, train_loader, criterion, optimizer, saved_dir, val_every, device)

Start training..
Epoch [1/30], Step [25/55], Loss: 0.9944
Epoch [1/30], Step [50/55], Loss: 0.9926
Start validation #1
Validation #1  Average Loss: 1.0611
Best performance at epoch: 1
Save model in ./saved/LSTM
Epoch [2/30], Step [25/55], Loss: 1.0302
Epoch [2/30], Step [50/55], Loss: 0.9569
Start validation #2
Validation #2  Average Loss: 0.9997
Best performance at epoch: 2
Save model in ./saved/LSTM
Epoch [3/30], Step [25/55], Loss: 0.9827
Epoch [3/30], Step [50/55], Loss: 0.9354
Start validation #3
Validation #3  Average Loss: 0.9314
Best performance at epoch: 3
Save model in ./saved/LSTM
Epoch [4/30], Step [25/55], Loss: 0.8337
Epoch [4/30], Step [50/55], Loss: 0.8170
Start validation #4
Validation #4  Average Loss: 0.8491
Best performance at epoch: 4
Save model in ./saved/LSTM
Epoch [5/30], Step [25/55], Loss: 0.7664
Epoch [5/30], Step [50/55], Loss: 0.6448
Start validation #5
Validation #5  Average Loss: 0.7396
Best performance at epoch: 5
Save model in ./saved/LSTM
Epoch [6/30],

### Load and Test Model

In [22]:
model_path = './saved/LSTM/best_model.pt'
model = SimpleLSTM().to(device) 

checkpoint = torch.load(model_path)    
state_dict = checkpoint['net']   
model.load_state_dict(state_dict) 

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [23]:
test(model, test_loader, criterion, baseline_loss, device)

Start test..
Test  Average Loss: 0.0832  Baseline Loss: 0.0852
Above baseline performance!


In [24]:
for i in range(15):
    data_idx = np.random.randint(len(test_data))
    sequence = test_data[data_idx][0]
    sequence = torch.Tensor(sequence).unsqueeze(0).to(device)
    #print(sequence.shape)
    pred = model(sequence)
    pred = pred.item() * test_data.std[0] + test_data.mean[0]
    
    target = test_data[data_idx][1][0] * test_data.std[0] + test_data.mean[0]
    print('Prediction: {:.1f} / Actual Temperature: {:.1f}'.format(pred, target))

Prediction: 19.9 / Actual Temperature: 21.4
Prediction: 21.1 / Actual Temperature: 14.6
Prediction: 17.4 / Actual Temperature: 21.3
Prediction: 11.6 / Actual Temperature: 18.6
Prediction: 14.3 / Actual Temperature: 8.9
Prediction: 16.9 / Actual Temperature: 11.5
Prediction: -2.8 / Actual Temperature: -4.9
Prediction: 17.5 / Actual Temperature: 22.3
Prediction: 17.8 / Actual Temperature: 12.8
Prediction: 9.1 / Actual Temperature: 11.7
Prediction: 19.6 / Actual Temperature: 22.6
Prediction: 23.5 / Actual Temperature: 25.3
Prediction: 15.1 / Actual Temperature: 16.7
Prediction: 22.9 / Actual Temperature: 21.8
Prediction: 19.2 / Actual Temperature: 21.6
